Import packages and processed run metrics

In [7]:
# packages
import os
import re
import numpy as np
import pandas as pd

# source
helper = __import__('00_helper')

# control
top_k = 1
with_ft = 1
write_output = 1

validation_metric = 'Results/val_acc'
project_names = [
    'femnist--s02', 'sst2', 'pubmed'
    #'cifar--alpha5.0', 'cifar--alpha0.5', 'cifar--alpha0.1'
]

df = helper.load_data(project_names)


# Custom sorts
from pandas.api.types import CategoricalDtype
cat_method_type = CategoricalDtype(
    [
        'ditto', 'fedbn', 'fedem', 'pfedme',  # pfl-methods
        'fomaml', 'fedavg', 'exact'  # ml-methodsd
    ],
    ordered=True
)
cat_dataset_type = CategoricalDtype(
    ['femnist--s02', 'sst2', 'pubmed'],
    ordered=True
)


input datasets:
	 (661, 36)
	 (1230, 44)
	 (790, 37)
all runs: (2681, 44)


In [8]:
def df_to_latex(
    df,
    id_columns,
    file_name=None,
    remove_columns=None,
    file_path='output'
):

    # remove columns not of interest
    if remove_columns is None:
        remove_columns = list()
    temp_df = df[[name for name in df.columns if name not in remove_columns]]

    # sort by id columns
    temp_df = temp_df.sort_values(by=id_columns)

    # create output file
    string_df = temp_df.to_string(
        header=True,
        index=False,
        index_names=False
    )

    # remove any leading space
    string_df = re.sub('^[\s]+', '', string_df)
    # remove leading space after newlines
    string_df = re.sub('\n[\s]+', '\n', string_df)
    # replace white space between words with table column skip
    string_df = re.sub('[ \t]+', ' & ', string_df)

    # add latex newline to end of each line
    string_df = string_df.replace('_', ' ')
    string_df = string_df.replace('\n', ' \\\\\n')
    string_df = string_df + ' \\\\'

    if file_name is not None:

        # write to file
        with open(os.path.join(file_path, f'{file_name}.txt'), 'w') as f:
            f.writelines(string_df)

    else:
        return string_df


Filter data to runs of interest
Apply summary function to filtered dataset

In [9]:
subset_df = df
subset_df['method'] = subset_df['method'].astype(cat_method_type)
subset_df['dataset'] = subset_df['dataset'].astype(cat_dataset_type)
print('input dataset:', df.shape)

## row (run) filtering
# remove extra hyper-parameter searches

subset_df = subset_df.loc[(subset_df.n_epochs < 6)]
print('\t reduce to 3 or fewer local update steps:', subset_df.shape)

subset_df = subset_df.loc[(subset_df.K < 6) | (subset_df.method != 'pfedme')]
print('\t reduce to 3 or fewer local meta-learning steps for pfedme:', subset_df.shape)

subset_df = subset_df.loc[((10 * subset_df.beta).astype('Int64') % 2 == 0) | (subset_df.method != 'exact')]
print('\t reduce beta grid for decay:', subset_df.shape)

## column (metric) filtering
subset_df = subset_df[[
    name for name in subset_df.columns
    if (
        not re.search('test', name)
        and not re.search('f1', name)
        and not re.search('loss', name)
    )
]]

def convert_SI(x):
    if pd.isna(x):
        return x

    SI = {
        'K': 10e3,
        'M': 10e6,
        'G': 10e9,
        'T': 10e12
    }

    units = re.sub('[0-9.]', '', x)
    x = float(re.sub('[^0-9.]', '', x))

    if units == '':
        return x
    return x * SI[units]


def to_SI(x, unit='G', decimals=2):
    SI = {
        'K': 10e3, 'M': 10e6,
        'G': 10e9, 'T': 10e12
    }

    assert unit in [name for name, _ in SI.items()]
    return str(round(x / SI[unit], decimals)) + f' {unit}'


#convert_SI('19.0M')

input dataset: (2681, 44)
	 reduce to 3 or fewer local update steps: (1995, 44)
	 reduce to 3 or fewer local meta-learning steps for pfedme: (1915, 44)
	 reduce beta grid for decay: (1818, 44)


In [10]:
# columns of interest
id_columns = ['dataset', 'method', validation_metric]
metrics = [name for name in subset_df.columns if re.search('^sys', name)]
print('metrics:', metrics)
object_metrics= [
    name for name, types in zip(subset_df.columns, subset_df.dtypes)
    if re.search('^sys', name)
    and types == 'object'
]

float_df = subset_df.loc[subset_df.finetune == with_ft]
float_df = float_df[id_columns + metrics]
float_df[object_metrics] = float_df[object_metrics].applymap(convert_SI).apply(pd.to_numeric)
float_df['sys_avg/total_bytes'] = float_df['sys_avg/total_download_bytes'] \
                          + float_df['sys_avg/total_upload_bytes']

# filter all non-zero
float_df = float_df.loc[
    (float_df['sys_avg/global_convergence_round'] > 0)
    & (float_df['sys_avg/global_convergence_time_minutes'] > 0)
    & (float_df['sys_avg/total_bytes'] > 0)
    & (float_df['sys_avg/total_flops'] > 0)
]

## get best runs for each group
print('before:', float_df.shape)
best_runs = float_df \
    .sort_values(by=validation_metric, ascending=False) \
    .groupby(['dataset', 'method']) \
    .head(top_k)
print('after:', best_runs.shape)


metrics: ['sys_avg/total_flops', 'sys_avg/total_model_size', 'sys_avg/local_convergence_time_minutes', 'sys_avg/global_convergence_round', 'sys_avg/total_upload_bytes', 'sys_avg/fl_end_time_minutes', 'sys_avg/local_convergence_round', 'sys_avg/global_convergence_time_minutes', 'sys_avg/total_download_bytes']
before: (725, 13)
after: (21, 13)


In [11]:
sub_metrics = [
    'sys_avg/global_convergence_round',
    'sys_avg/global_convergence_time_minutes',
    'sys_avg/total_bytes',
    'sys_avg/total_flops'
]
id_columns = ['method', 'dataset']
avg_runs = best_runs.groupby(id_columns)[sub_metrics].mean()

avg_runs['sys_avg/global_convergence_round'] = avg_runs['sys_avg/global_convergence_round'].apply(round, ndigits=2)
avg_runs['sys_avg/global_convergence_time_minutes'] = avg_runs['sys_avg/global_convergence_time_minutes'].apply(round, ndigits=2)
avg_runs['sys_avg/total_bytes'] = avg_runs['sys_avg/total_bytes'].apply(to_SI, unit='M')
avg_runs['sys_avg/total_flops'] = avg_runs['sys_avg/total_flops'].apply(to_SI, unit='G')
avg_runs = avg_runs.add_suffix('_mean').reset_index()

avg_runs.columns = [re.sub('sys_avg/', '', name) for name in avg_runs.columns]
if write_output:
    df_to_latex(avg_runs, id_columns, 'method_costs--ft_' + ('yes' if with_ft else 'no'))

In [12]:
avg_runs.head(6)

,method,dataset,global_convergence_round_mean,global_convergence_time_minutes_mean,total_bytes_mean,total_flops_mean
0,ditto,femnist--s02,420.0,85.00,2.02 M,11.86 G
1,ditto,sst2,30.0,26.65,0.44 M,1010.0 G
2,ditto,pubmed,110.0,0.86,3.14 M,80.99 G
3,fedbn,femnist--s02,450.0,39.24,1.78 M,270.18 G
4,fedbn,sst2,85.0,15.36,1.24 M,1110.0 G
5,fedbn,pubmed,50.0,0.33,1.08 M,32.06 G
